# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
query = "CREATE KEYSPACE music WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 3};"

try:
    session.execute(query)
except Exception as e:
    print(e)

Keyspace 'music' already exists


#### Set Keyspace

In [7]:
query = "USE music"

try:
    session.execute(query)
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Following table is created to fetch artist, song title and song's length from music app history
# that was heard for a given session and iteminsession.
# Primary Keys are set as - sessionId, itemInSession
query =  "CREATE TABLE IF NOT EXISTS music_by_sessionid_itemsinsession( \
             sessionId int \
            ,itemInSession int \
            ,artist text \
            ,song text \
            ,length double \
            ,PRIMARY KEY(sessionId, itemInSession) \
          ) WITH COMMENT = 'Find artist, song and song length by Session Id'\
        "
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# Use the genrated event datafile as the source
file = 'event_datafile_new.csv'

# Used for referencing the correct cell
#"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"
#0        ,1          ,2        ,3              ,4         ,5        ,6     ,7         ,8         ,9    ,10

#insert the appropriate cell data into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert the data in the same order of the CREATE table statement
        query = "INSERT INTO music_by_sessionid_itemsinsession(sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Verify the inserted data by a specific use case
query = "SELECT artist, song, length FROM music_by_sessionid_itemsinsession WHERE sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
    ## execute and run all the rows
    for row in rows:
        print(row.artist,row.song, row.length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [11]:
# Following table is created to fetch artist, song title and user details from music app history
# that was heard for a given userId and SessionId.
# Primary Keys are set as - userId, sessionId
# Clustering Keys are set as - itemInSession
query =  "CREATE TABLE IF NOT EXISTS music_by_userid_sessionid( \
             userId int \
            ,sessionId int \
            ,itemInSession int \
            ,artist text \
            ,song text \
            ,name text \
            ,PRIMARY KEY((userId, sessionId),itemInSession) \
          ) WITH COMMENT = 'Find artist name and song by user id and session id'\
        "
try:
    session.execute(query)
except Exception as e:
    print(e)

# use the generated event datafile as source of data
file = 'event_datafile_new.csv'

# Used for referencing the correct cell
#"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"
#0        ,1          ,2        ,3              ,4         ,5        ,6     ,7         ,8         ,9    ,10


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert the data in same order as CREATE table statement
        query = "INSERT INTO music_by_userid_sessionid(userId, sessionId, itemInSession, artist, song, name)"
        query = query + " VALUES(%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + ' ' + line[4]))    

# Verify inserted data for a specific user case
query = "SELECT artist, song, name, itemInSession FROM music_by_userid_sessionid WHERE userId = 10 and sessionId = 182 order by itemInSession"

try:
    rows = session.execute(query)
    ## execute and run all the rows
    for row in rows:
        print(row.artist,row.song, row.name)
except Exception as e:
    print(e)        

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [12]:
# Following table is created to fetch every user in app history who has listend to a specific song
# Primary Keys are set as - song, userId
query =  "CREATE TABLE IF NOT EXISTS music_by_song( \
             song text \
            ,userId int \
            ,name text \
            ,PRIMARY KEY (song, userId) \
            ) WITH COMMENT = 'Get user name from song title'\
        "
try:
    session.execute(query)
except Exception as e:
    print(e)

#use the generated event datafile as source
file = 'event_datafile_new.csv'

# Used for referencing the correct cell
#"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"
#0        ,1          ,2        ,3              ,4         ,5        ,6     ,7         ,8         ,9    ,10

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert the data in same order of CREATE table.
        query = "INSERT INTO music_by_song(song, userId, name)"
        query = query + " VALUES(%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1] + ' ' + line[4]))  

#Verify inserted data for a specific use case
query = "SELECT name FROM music_by_song WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
    ## execute and run all the rows
    for row in rows:
        print(row.name)
except Exception as e:
    print(e)  
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [13]:
## Drop table music_by_sessionid_itemsinsession
query =  "DROP TABLE IF EXISTS music_by_sessionid_itemsinsession;"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
## Drop table music_by_song
query =  "DROP TABLE IF EXISTS music_by_song;"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Drop table music_by_userid_sessionid
query =  "DROP TABLE IF EXISTS music_by_userid_sessionid;"
try:
    session.execute(query)
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()